# Instalação das bibliotecas utilizadas

Antes de iniciarmos a análise dos dados de Consumo no Google Colab, é essencial garantir que todas as bibliotecas necessárias estejam devidamente instaladas e atualizadas. No ambiente do Colab, embora muitas bibliotecas já venham pré-instaladas, pode ser necessário instalar ou atualizar pacotes adicionais para garantir que o código funcione corretamente, como por exemplo o MeteoStat, biblioteca utilizada para coletar os dados climáticos.

In [ ]:
!pip install meteostat

In [ ]:
# Importação das bibliotecas

import numpy as np
import pandas as pd
import gdown
import plotly.graph_objects as go
from meteostat import Point, Daily
from datetime import datetime

In [ ]:
# Define o nome base para salvar os arquivos baixados
arquivo_destino_base = "dataset_{}.csv"

# IDs dos arquivos no Google Drive, organizados por ano
ids = {
    "consumo_2024": "1-2YBshCNQI4pOMV83BGJnTrCFpDTcv_h",
    "consumo_2023": "1WmLtlgEgMvAqJ9kjxP_xT0oecsLaM_vr",
    "consumo_2022": "1-8l0qgFq40NhqZbMM_gBYJO32vSbr0eG",
    "consumo_2021": "12-fFTkYIDOIorDnj3zAWwynNPC6B8mb2",
    "consumo_2020": "1-1pOoa0eJlNJ94BMi7p4PTx5KUS96mhX",
    "consumo_2019": "1MRCFBXGPkdkEe546LbrOw3ypJRKi4ofG",
}

# Dicionário para armazenar os DataFrames
dataframes = {}

# Baixa e lê cada arquivo, salvando-o em um DataFrame
for key, file_id in ids.items():
    url = f"https://drive.google.com/uc?id={file_id}"
    arquivo_destino = arquivo_destino_base.format(key)

    gdown.download(url, arquivo_destino, quiet=False)
    df = pd.read_csv(arquivo_destino, sep=";")
    dataframes[key] = df

# Consumo 2019

## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2019 e os dados climáticos, que serão utilizados.

In [ ]:
df_19 = pd.read_csv("/content/dataset_consumo_2019.csv", delimiter=",")

In [ ]:
df_19

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define o intervalo de datas
start = datetime(2019, 1, 1)
end = datetime(2019, 12, 31)

# Busca os dados diários de clima para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados climáticos
df_monthly_weather_19 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para termos mais descritivos em português
df_weather_19 = df_monthly_weather_19.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_19

A seguir iremos remover as colunas que julgamos não serem importantes para entender a mudança do consumo da água.

In [ ]:
# Remove as colunas desnecessárias do DataFrame
df_weather_19 = df_weather_19.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])
df_weather_19

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime, garantindo que as datas sejam reconhecidas corretamente
df_19['REFERENCIA'] = pd.to_datetime(df_19['REFERENCIA'])

# Cria uma nova coluna 'MES' extraindo o número do mês da coluna 'REFERENCIA'
df_19['MES'] = df_19['REFERENCIA'].dt.month

# Extrai o número do mês do índice (assumindo que o índice do DataFrame é uma coluna datetime) e cria uma nova coluna 'MES'
df_weather_19['MES'] = df_weather_19.index.month

# Agrupa os dados por mês e calcula a média do consumo medido ('CONS_MEDIDO') para cada mês
consumo_medio_19 = df_19.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina os dados de consumo médio com os dados meteorológicos com base na coluna 'MES'
df_combined_19 = pd.merge(consumo_medio_19, df_weather_19, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_19'
linhas, colunas = df_19.shape

print(f"O DataFrame de 2019 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Extrai os valores únicos da coluna 'MATRICULA' no DataFrame 'df_19'
matriculas_unicas = df_19['MATRICULA'].unique()

# Calcula o número total de valores únicos na coluna 'MATRICULA'
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2019 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total dos valores na coluna 'CONS_MEDIDO' no DataFrame 'df_19'
soma_consumo = df_19['CONS_MEDIDO'].sum()

print(f"Em 2019 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses de 2019. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona o consumo médio mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_19['MES'],
    y=df_combined_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a temperatura média mensal ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_19['MES'],
    y=df_combined_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlation_19 = df_combined_19['CONS_MEDIDO'].corr(df_combined_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_19:.2f}")

### Consumo Médio Mensal e Temperatura Média por Categoria

In [ ]:
# Obtém os valores únicos da coluna 'CATEGORIA' no DataFrame 'df_19'
categorias_unicas = df_19['CATEGORIA'].unique()

categorias_unicas

Este gráfico apresenta a análise do consumo médio de água na categoria **Pública** em relação à temperatura média ao longo dos meses de 2019.

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_19 = df_19[df_19['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_19 = consumo_publica_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_19 = pd.merge(consumo_medio_publica_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_19['MES'],
    y=df_combined_publica_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_19['MES'],
    y=df_combined_publica_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Pública'
correlacao_pearson = df_combined_publica_19['CONS_MEDIDO'].corr(df_combined_publica_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.2f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Comercial** em relação à temperatura média ao longo dos meses de 2019.

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_19 = df_19[df_19['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_19 = consumo_comercial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_19 = pd.merge(consumo_medio_comercial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_19['MES'],
    y=df_combined_comercial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_19['MES'],
    y=df_combined_comercial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Comercial'
correlacao_pearson = df_combined_comercial_19['CONS_MEDIDO'].corr(df_combined_comercial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Residencial** em relação à temperatura média ao longo dos meses de 2019.

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_19 = df_19[df_19['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_19 = consumo_residencial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_19 = pd.merge(consumo_medio_residencial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_19['MES'],
    y=df_combined_residencial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_19['MES'],
    y=df_combined_residencial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Residencial'
correlacao_pearson = df_combined_residencial_19['CONS_MEDIDO'].corr(df_combined_residencial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

Este gráfico apresenta a análise do consumo médio de água na categoria **Industrial** em relação à temperatura média ao longo dos meses de 2019.

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_19 = df_19[df_19['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_19 = consumo_industrial_19.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_19 = pd.merge(consumo_medio_industrial_19, df_weather_19, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_19['MES'],
    y=df_combined_industrial_19['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_19['MES'],
    y=df_combined_industrial_19['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média para a categoria 'Industrial'
correlacao_pearson = df_combined_industrial_19['CONS_MEDIDO'].corr(df_combined_industrial_19['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

Nesta seção, apresentamos uma análise detalhada do consumo médio mensal de água para diferentes categorias: Pública, Comercial, Residencial e Industrial. Ao comparar essas categorias lado a lado, podemos identificar padrões de consumo específicos, sazonalidades e variações que refletem as diferentes necessidades e comportamentos de cada grupo ao longo do ano de 2019.


In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_19[df_19['CATEGORIA'] == categoria]
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação

Nesta seção, exploramos o consumo médio mensal de água segmentado por tipo de ligação: Hidrometrado e Consumo Fixo. Essa análise permite observar como o consumo varia ao longo do ano para cada tipo de ligação.


In [ ]:
# Obtém os valores únicos da coluna 'TIPO_LIGACAO' no DataFrame 'df_19'
tipo_ligacao = df_19['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_19[df_19['TIPO_LIGACAO'] == tipo]
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

Nesta seção, analisamos o consumo médio mensal de água segmentado por subcategoria. As subcategorias incluem Pública, Comercial, Residencial, Residencial Social, Industrial e Residencial Indubrasil.


In [ ]:
# Obtém os valores únicos da coluna 'SUB_CATEGORIA' no DataFrame 'df_19'
sub_categoria = df_19['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_19[df_19['SUB_CATEGORIA'] == sub_categoria]
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2019',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
df_res_indubrasil = df_19[df_19['SUB_CATEGORIA'] == 'RES. INDUBRASIL']
df_res_indubrasil

# Consumo 2020

## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2020 e os dados climáticos, que serão utilizados.

In [ ]:
df_20 = pd.read_csv("/content/dataset_consumo_2020.csv", delimiter=",")

In [ ]:
df_20

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define as datas de início e fim
start = datetime(2020, 1, 1)
end = datetime(2020, 12, 31)

# Busca os dados diários para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados diários
df_monthly_weather_20 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para nomes mais descritivos
df_weather_20 = df_monthly_weather_20.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_20

In [ ]:
# Remove as colunas desnecessárias do DataFrame
df_weather_20 = df_weather_20.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])

df_weather_20

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime
df_20['REFERENCIA'] = pd.to_datetime(df_20['REFERENCIA'])

# Extrai o mês da coluna 'REFERENCIA' e cria a coluna 'MES'
df_20['MES'] = df_20['REFERENCIA'].dt.month

# Extrai o mês do índice do DataFrame de clima e cria a coluna 'MES'
df_weather_20['MES'] = df_weather_20.index.month

# Calcula o consumo médio mensal e reseta o índice
consumo_medio_20 = df_20.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina o consumo médio mensal com os dados climáticos com base na coluna 'MES'
df_combined_20 = pd.merge(consumo_medio_20, df_weather_20, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados.

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_20'
linhas, colunas = df_20.shape

print(f"O DataFrame de 2020 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Obtém os valores únicos da coluna 'MATRICULA' no DataFrame 'df_20'
matriculas_unicas = df_20['MATRICULA'].unique()

# Calcula o número total de matrículas únicas
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2020 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total do consumo medido no DataFrame 'df_20'
soma_consumo = df_20['CONS_MEDIDO'].sum()

print(f"Em 2020 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses de 2020. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_20['MES'],
    y=df_combined_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_20['MES'],
    y=df_combined_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2020',  # Título do gráfico
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),  # Configuração do eixo x
    yaxis=dict(title='Consumo Médio'),  # Configuração do primeiro eixo y
    yaxis2=dict(  # Configuração do segundo eixo y (temperatura)
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,  # Posiciona a legenda ao centro
        y=-0.2,  # Posiciona a legenda abaixo do gráfico
        orientation="h",
        xanchor="center"
    ),
    width=800,  # Define a largura do gráfico
    height=500  # Define a altura do gráfico
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlation_20 = df_combined_20['CONS_MEDIDO'].corr(df_combined_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_20:.2f}")

### Consumo Médio Mensal e Temperatura Média por Categoria

In [ ]:
categorias_unicas = df_20['CATEGORIA'].unique()
categorias_unicas

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_20 = df_20[df_20['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_20 = consumo_publica_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_20 = pd.merge(consumo_medio_publica_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_20['MES'],
    y=df_combined_publica_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_20['MES'],
    y=df_combined_publica_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_publica_20['CONS_MEDIDO'].corr(df_combined_publica_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.4f}")

In [ ]:
consumo_comercial_20 = df_20[df_20['CATEGORIA'] == 'COMERCIAL']

consumo_medio_comercial_20 = consumo_comercial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

df_combined_comercial_20 = pd.merge(consumo_medio_comercial_20, df_weather_20, on='MES')

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_20 = df_20[df_20['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_20 = consumo_comercial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_20 = pd.merge(consumo_medio_comercial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_20['MES'],
    y=df_combined_comercial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_comercial_20['CONS_MEDIDO'].corr(df_combined_comercial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_20 = df_20[df_20['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_20 = consumo_residencial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_20 = pd.merge(consumo_medio_residencial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_20['MES'],
    y=df_combined_residencial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_20['MES'],
    y=df_combined_residencial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_residencial_20['CONS_MEDIDO'].corr(df_combined_residencial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_20 = df_20[df_20['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_20 = consumo_industrial_20.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_20 = pd.merge(consumo_medio_industrial_20, df_weather_20, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_20['MES'],
    y=df_combined_industrial_20['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_20['MES'],
    y=df_combined_industrial_20['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_industrial_20['CONS_MEDIDO'].corr(df_combined_industrial_20['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

Nesta seção, apresentamos uma análise detalhada do consumo médio mensal de água para diferentes categorias: Pública, Comercial, Residencial e Industrial. Ao comparar essas categorias lado a lado, podemos identificar padrões de consumo específicos, sazonalidades e variações que refletem as diferentes necessidades e comportamentos de cada grupo ao longo do ano de 2020.


In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_20[df_20['CATEGORIA'] == categoria]  # Filtra os dados pela categoria
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação

Nesta seção, exploramos o consumo médio mensal de água segmentado por tipo de ligação: Hidrometrado e Consumo Fixo. Essa análise permite observar como o consumo varia ao longo do ano para cada tipo de ligação.


In [ ]:
# Obtém os valores únicos da coluna 'TIPO_LIGACAO' no DataFrame 'df_19'
tipo_ligacao = df_20['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_20[df_20['TIPO_LIGACAO'] == tipo]  # Filtra os dados pelo tipo de ligação
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

Nesta seção, analisamos o consumo médio mensal de água segmentado por subcategoria. As subcategorias incluem Pública, Comercial, Residencial, Residencial Social, Industrial e Residencial Indubrasil.


In [ ]:
sub_categoria = df_20['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_20[df_20['SUB_CATEGORIA'] == sub_categoria]  # Filtra os dados pela subcategoria
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2020',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

# Consumo 2021


## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2021 e os dados climáticos, que serão utilizados.

In [ ]:
df_21 = pd.read_csv("/content/dataset_consumo_2021.csv", delimiter=",")

In [ ]:
df_21

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define as datas de início e fim
start = datetime(2021, 1, 1)
end = datetime(2021, 12, 31)

# Busca os dados diários para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados diários
df_monthly_weather_21 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para nomes mais descritivos
df_weather_21 = df_monthly_weather_21.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_21

In [ ]:
df_weather_21 = df_weather_21.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])

df_weather_21

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime
df_21['REFERENCIA'] = pd.to_datetime(df_21['REFERENCIA'])

# Extrai o mês da coluna 'REFERENCIA' e cria a coluna 'MES'
df_21['MES'] = df_21['REFERENCIA'].dt.month

# Extrai o mês do índice do DataFrame de clima e cria a coluna 'MES'
df_weather_21['MES'] = df_weather_21.index.month

# Calcula o consumo médio mensal e reseta o índice
consumo_medio_21 = df_21.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina o consumo médio mensal com os dados climáticos com base na coluna 'MES'
df_combined_21 = pd.merge(consumo_medio_21, df_weather_21, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados.

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_21'
linhas, colunas = df_21.shape

print(f"O DataFrame de 2021 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Obtém os valores únicos da coluna 'MATRICULA' no DataFrame 'df_21'
matriculas_unicas = df_21['MATRICULA'].unique()

# Calcula o número total de matrículas únicas
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2021 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total do consumo medido no DataFrame 'df_21'
soma_consumo = df_21['CONS_MEDIDO'].sum()

print(f"Em 2021 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses de 2021. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_21['MES'],
    y=df_combined_21['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_21['MES'],
    y=df_combined_21['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlation_21 = df_combined_21['CONS_MEDIDO'].corr(df_combined_21['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_21:.2f}")

### Consumo Médio Mensal e Temperatura Média por Categoria

In [ ]:
categorias_unicas = df_21['CATEGORIA'].unique()
categorias_unicas

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_21 = df_21[df_21['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_21 = consumo_publica_21.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_21 = pd.merge(consumo_medio_publica_21, df_weather_21, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_21['MES'],
    y=df_combined_publica_21['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_21['MES'],
    y=df_combined_publica_21['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_publica_21['CONS_MEDIDO'].corr(df_combined_publica_21['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_21 = df_21[df_21['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_21 = consumo_comercial_21.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_21 = pd.merge(consumo_medio_comercial_21, df_weather_21, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_21['MES'],
    y=df_combined_comercial_21['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_21['MES'],
    y=df_combined_comercial_21['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_comercial_21['CONS_MEDIDO'].corr(df_combined_comercial_21['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_21 = df_21[df_21['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_21 = consumo_residencial_21.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_21 = pd.merge(consumo_medio_residencial_21, df_weather_21, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_21['MES'],
    y=df_combined_residencial_21['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_21['MES'],
    y=df_combined_residencial_21['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_residencial_21['CONS_MEDIDO'].corr(df_combined_residencial_21['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_21 = df_21[df_21['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_21 = consumo_industrial_21.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_21 = pd.merge(consumo_medio_industrial_21, df_weather_21, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_21['MES'],
    y=df_combined_industrial_21['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_21['MES'],
    y=df_combined_industrial_21['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
# Calcula a correlação de Pearson entre o consumo médio e a temperatura média
correlacao_pearson = df_combined_industrial_21['CONS_MEDIDO'].corr(df_combined_industrial_21['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_21[df_21['CATEGORIA'] == categoria]  # Filtra os dados pela categoria
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação


In [ ]:
tipo_ligacao = df_21['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_21[df_21['TIPO_LIGACAO'] == tipo]
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

In [ ]:
sub_categoria = df_21['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_21[df_21['SUB_CATEGORIA'] == sub_categoria]  #
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2021',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

# Consumo 2022


## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2022 e os dados climáticos, que serão utilizados.

In [ ]:
df_22 = pd.read_csv("/content/dataset_consumo_2022.csv", delimiter=",")

In [ ]:
df_22

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define as datas de início e fim
start = datetime(2022, 1, 1)
end = datetime(2022, 12, 31)

# Busca os dados diários para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados diários
df_monthly_weather_22 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para nomes mais descritivos
df_weather_22 = df_monthly_weather_22.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_22

In [ ]:
df_weather_22 = df_weather_22.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])

In [ ]:
df_weather_22

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime
df_22['REFERENCIA'] = pd.to_datetime(df_22['REFERENCIA'])

# Extrai o mês da coluna 'REFERENCIA' e cria a coluna 'MES'
df_22['MES'] = df_22['REFERENCIA'].dt.month

# Extrai o mês do índice do DataFrame de clima e cria a coluna 'MES'
df_weather_22['MES'] = df_weather_22.index.month

# Calcula o consumo médio mensal e reseta o índice
consumo_medio_22 = df_22.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina o consumo médio mensal com os dados climáticos com base na coluna 'MES'
df_combined_22 = pd.merge(consumo_medio_22, df_weather_22, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados.

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_22'
linhas, colunas = df_22.shape

print(f"O DataFrame de 2022 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Obtém os valores únicos da coluna 'MATRICULA' no DataFrame 'df_22'
matriculas_unicas = df_22['MATRICULA'].unique()

# Calcula o número total de matrículas únicas
tamanho = len(matriculas_unicas)

# Exibe a quantidade de matrículas únicas no DataFrame de 2022
print(f"O DataFrame de 2022 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total do consumo medido no DataFrame 'df_22'
soma_consumo = df_22['CONS_MEDIDO'].sum()

# Exibe o total de consumo em metros cúbicos (m^3) para o ano de 2022
print(f"Em 2022 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_22 = df_22[df_22['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_22 = consumo_publica_22.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_22 = pd.merge(consumo_medio_publica_22, df_weather_22, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_22['MES'],
    y=df_combined_publica_22['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_22['MES'],
    y=df_combined_publica_22['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlation_22 = df_combined_22['CONS_MEDIDO'].corr(df_combined_22['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_22:.2f}")

### Consumo Médio Mensal e Temperatura Média

Este gráfico ilustra a relação entre o consumo médio de água e a temperatura média ao longo dos meses de 2022. A linha verde tracejada representa o consumo médio mensal, enquanto a linha vermelha sólida mostra a variação da temperatura média mensal.


In [ ]:
categorias_unicas = df_22['CATEGORIA'].unique()
categorias_unicas

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_22 = df_22[df_22['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_22 = consumo_publica_22.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_22 = pd.merge(consumo_medio_publica_22, df_weather_22, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_22['MES'],
    y=df_combined_publica_22['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_22['MES'],
    y=df_combined_publica_22['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_publica_22['CONS_MEDIDO'].corr(df_combined_publica_22['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_22 = df_22[df_22['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_22 = consumo_comercial_22.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_22 = pd.merge(consumo_medio_comercial_22, df_weather_22, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_22['MES'],
    y=df_combined_comercial_22['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_22['MES'],
    y=df_combined_comercial_22['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_comercial_22['CONS_MEDIDO'].corr(df_combined_comercial_22['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_22 = df_22[df_22['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_22 = consumo_residencial_22.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_22 = pd.merge(consumo_medio_residencial_22, df_weather_22, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_22['MES'],
    y=df_combined_residencial_22['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_22['MES'],
    y=df_combined_residencial_22['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_residencial_22['CONS_MEDIDO'].corr(df_combined_residencial_22['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_22 = df_22[df_22['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_22 = consumo_industrial_22.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_22 = pd.merge(consumo_medio_industrial_22, df_weather_22, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_22['MES'],
    y=df_combined_industrial_22['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_22['MES'],
    y=df_combined_industrial_22['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_industrial_22['CONS_MEDIDO'].corr(df_combined_industrial_22['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_22[df_22['CATEGORIA'] == categoria]
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação


In [ ]:
tipo_ligacao = df_22['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_22[df_22['TIPO_LIGACAO'] == tipo]
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

In [ ]:
sub_categoria = df_22['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_22[df_22['SUB_CATEGORIA'] == sub_categoria]
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2022',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

# Consumo 2023


## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2023 e os dados climáticos, que serão utilizados.

In [ ]:
df_23 = pd.read_csv("/content/dataset_consumo_2023.csv", delimiter=",")

In [ ]:
df_23

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define as datas de início e fim
start = datetime(2023, 1, 1)
end = datetime(2023, 12, 31)

# Busca os dados diários para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados diários
df_monthly_weather_23 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para nomes mais descritivos
df_weather_23 = df_monthly_weather_23.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_23

In [ ]:
# Remove as colunas que não são necessárias para a análise
df_weather_23 = df_weather_23.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])

# Exibe o DataFrame após a remoção das colunas
df_weather_23

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime
df_23['REFERENCIA'] = pd.to_datetime(df_23['REFERENCIA'])

# Extrai o mês da coluna 'REFERENCIA' e cria a coluna 'MES'
df_23['MES'] = df_23['REFERENCIA'].dt.month

# Extrai o mês do índice do DataFrame de clima e cria a coluna 'MES'
df_weather_23['MES'] = df_weather_23.index.month

# Calcula o consumo médio mensal e reseta o índice
consumo_medio_23 = df_23.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina o consumo médio mensal com os dados climáticos com base na coluna 'MES'
df_combined_23 = pd.merge(consumo_medio_23, df_weather_23, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados.

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_23'
linhas, colunas = df_23.shape

print(f"O DataFrame de 2023 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Obtém os valores únicos da coluna 'MATRICULA' no DataFrame 'df_23'
matriculas_unicas = df_23['MATRICULA'].unique()

# Calcula o número total de matrículas únicas
tamanho = len(matriculas_unicas)

print(f"O DataFrame de 2023 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total do consumo medido no DataFrame 'df_22'
soma_consumo = df_23['CONS_MEDIDO'].sum()
print(f"Em 2023 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_23['MES'],
    y=df_combined_23['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_23['MES'],
    y=df_combined_23['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlation_23 = df_combined_23['CONS_MEDIDO'].corr(df_combined_23['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_23:.2f}")

### Consumo Médio Mensal e Temperatura Média por Categoria

In [ ]:
categorias_unicas = df_23['CATEGORIA'].unique()
categorias_unicas

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_23 = df_23[df_23['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_23 = consumo_publica_23.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_23 = pd.merge(consumo_medio_publica_23, df_weather_23, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_23['MES'],
    y=df_combined_publica_23['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_23['MES'],
    y=df_combined_publica_23['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_publica_23['CONS_MEDIDO'].corr(df_combined_publica_23['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_23 = df_23[df_23['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_23 = consumo_comercial_23.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_23 = pd.merge(consumo_medio_comercial_23, df_weather_23, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_23['MES'],
    y=df_combined_comercial_23['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_23['MES'],
    y=df_combined_comercial_23['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_comercial_23['CONS_MEDIDO'].corr(df_combined_comercial_23['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_23 = df_23[df_23['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_23 = consumo_residencial_23.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_23 = pd.merge(consumo_medio_residencial_23, df_weather_23, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_23['MES'],
    y=df_combined_residencial_23['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_23['MES'],
    y=df_combined_residencial_23['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_residencial_23['CONS_MEDIDO'].corr(df_combined_residencial_23['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_23 = df_23[df_23['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_23 = consumo_industrial_23.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_23 = pd.merge(consumo_medio_industrial_23, df_weather_23, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_23['MES'],
    y=df_combined_industrial_23['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_23['MES'],
    y=df_combined_industrial_23['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_industrial_23['CONS_MEDIDO'].corr(df_combined_industrial_23['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_23[df_23['CATEGORIA'] == categoria]
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação


In [ ]:
tipo_ligacao = df_23['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_23[df_23['TIPO_LIGACAO'] == tipo]  # Filtra os dados pelo tipo de ligação
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

In [ ]:
sub_categoria = df_23['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_23[df_23['SUB_CATEGORIA'] == sub_categoria]  # Filtra os dados pela subcategoria
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2023',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

# Consumo 2024


## Estruturação dos datasets

Nesta seção, iremos organizar os arquivos, dataset do Consumo 2024 e os dados climáticos, que serão utilizados.

In [ ]:
df_24 = pd.read_csv("/content/dataset_consumo_2024.csv", delimiter=",")

In [ ]:
df_24

Para o código a seguir é importante ressaltar que a Latitude e a Longitude foram retiradas de: [Campo Grande MS](https://pt.db-city.com/Brasil--Mato-Grosso-do-Sul--Campo-Grande)

In [ ]:
# Define a localização geográfica
location = Point(-20.4435, -54.6478)

# Define as datas de início e fim
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

# Busca os dados diários para a localização e período definidos
data = Daily(location, start, end)
data = data.fetch()

# Calcula a média mensal dos dados diários
df_monthly_weather_24 = data.resample('M').mean()

In [ ]:
# Renomeia as colunas do DataFrame para nomes mais descritivos
df_weather_24 = df_monthly_weather_24.rename(columns={
    'tavg': 'Temperatura Média',
    'tmin': 'Temperatura Mínima',
    'tmax': 'Temperatura Máxima',
    'prcp': 'Precipitação',
    'snow': 'Neve',
    'wdir': 'Direção do Vento',
    'wspd': 'Velocidade do Vento',
    'wpgt': 'Rajada Máxima de Vento',
    'pres': 'Pressão Atmosférica',
    'tsun': 'Insolação'
})

# Exibe o DataFrame renomeado
df_weather_24

In [ ]:
df_weather_24 = df_weather_24.drop(columns=['Neve', 'Precipitação', 'Direção do Vento', 'Velocidade do Vento', 'Rajada Máxima de Vento', 'Pressão Atmosférica', 'Insolação'])

df_weather_24

In [ ]:
# Converte a coluna 'REFERENCIA' para o tipo datetime
df_24['REFERENCIA'] = pd.to_datetime(df_24['REFERENCIA'])

# Extrai o mês da coluna 'REFERENCIA' e cria a coluna 'MES'
df_24['MES'] = df_24['REFERENCIA'].dt.month

# Extrai o mês do índice do DataFrame de clima e cria a coluna 'MES'
df_weather_24['MES'] = df_weather_24.index.month

# Calcula o consumo médio mensal e reseta o índice
consumo_medio_24 = df_24.groupby('MES')['CONS_MEDIDO'].mean().reset_index()

# Combina o consumo médio mensal com os dados climáticos com base na coluna 'MES'
df_combined_24 = pd.merge(consumo_medio_24, df_weather_24, on='MES')

## Big Numbers

Nesta seção, apresentamos os alguns indicadores e métricas do nosso dataset, destacando valores significativos que fornecem uma visão rápida e intuitiva sobre o comportamento dos dados.

In [ ]:
# Obtém o número de linhas e colunas do DataFrame 'df_24'
linhas, colunas = df_24.shape

print(f"O DataFrame de 2024 tem {linhas} linhas e {colunas} colunas.")

In [ ]:
# Obtém os valores únicos da coluna 'MATRICULA' no DataFrame 'df_24'
matriculas_unicas = df_24['MATRICULA'].unique()

# Calcula o número total de matrículas únicas
tamanho = len(matriculas_unicas)

# Exibe a quantidade de matrículas únicas no DataFrame de 2024
print(f"O DataFrame de 2024 tem {tamanho} matrículas únicas")

In [ ]:
# Calcula a soma total do consumo medido no DataFrame 'df_24'
soma_consumo = df_24['CONS_MEDIDO'].sum()
print(f"Em 2024 foi consumido {soma_consumo} m^3")

## Gráficos

Nesta seção, apresentamos uma série de visualizações que ilustram as principais tendências e padrões encontrados nos dados. Os gráficos fornecem uma maneira intuitiva de entender a distribuição, variação e correlação entre as diferentes variáveis analisadas.


### Consumo Médio Mensal e Temperatura Média

In [ ]:
# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_24['MES'],
    y=df_combined_24['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média ao gráfico
fig.add_trace(go.Scatter(
    x=df_combined_24['MES'],
    y=df_combined_24['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 9))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlation_24 = df_combined_24['CONS_MEDIDO'].corr(df_combined_24['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média: {correlation_24:.2f}")

### Consumo Médio Mensal e Temperatura Média por Categoria

In [ ]:
categorias_unicas = df_24['CATEGORIA'].unique()
categorias_unicas

In [ ]:
# Filtra os dados para a categoria 'PUBLICA'
consumo_publica_24 = df_24[df_24['CATEGORIA'] == 'PUBLICA']

# Calcula o consumo médio mensal para a categoria 'PUBLICA'
consumo_medio_publica_24 = consumo_publica_24.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_publica_24 = pd.merge(consumo_medio_publica_24, df_weather_24, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'PUBLICA'
fig.add_trace(go.Scatter(
    x=df_combined_publica_24['MES'],
    y=df_combined_publica_24['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Pública)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_publica_24['MES'],
    y=df_combined_publica_24['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Pública - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 9))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_publica_24['CONS_MEDIDO'].corr(df_combined_publica_24['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Pública: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'COMERCIAL'
consumo_comercial_24 = df_24[df_24['CATEGORIA'] == 'COMERCIAL']

# Calcula o consumo médio mensal para a categoria 'COMERCIAL'
consumo_medio_comercial_24 = consumo_comercial_24.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_comercial_24 = pd.merge(consumo_medio_comercial_24, df_weather_24, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'COMERCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_comercial_24['MES'],
    y=df_combined_comercial_24['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Comercial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_comercial_24['MES'],
    y=df_combined_comercial_24['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Comercial - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_comercial_24['CONS_MEDIDO'].corr(df_combined_comercial_24['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Comercial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'RESIDENCIAL'
consumo_residencial_24 = df_24[df_24['CATEGORIA'] == 'RESIDENCIAL']

# Calcula o consumo médio mensal para a categoria 'RESIDENCIAL'
consumo_medio_residencial_24 = consumo_residencial_24.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_residencial_24 = pd.merge(consumo_medio_residencial_24, df_weather_24, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'RESIDENCIAL'
fig.add_trace(go.Scatter(
    x=df_combined_residencial_24['MES'],
    y=df_combined_residencial_24['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Residencial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_residencial_24['MES'],
    y=df_combined_residencial_24['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Residencial - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_residencial_24['CONS_MEDIDO'].corr(df_combined_residencial_24['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Residencial: {correlacao_pearson:.4f}")

In [ ]:
# Filtra os dados para a categoria 'INDUSTRIAL'
consumo_industrial_24 = df_24[df_24['CATEGORIA'] == 'INDUSTRIAL']

# Calcula o consumo médio mensal para a categoria 'INDUSTRIAL'
consumo_medio_industrial_24 = consumo_industrial_24.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

# Combina o consumo médio mensal com os dados de temperatura média com base no mês
df_combined_industrial_24 = pd.merge(consumo_medio_industrial_24, df_weather_24, on='MES')

# Cria uma figura para o gráfico
fig = go.Figure()

# Adiciona a linha do consumo médio para a categoria 'INDUSTRIAL'
fig.add_trace(go.Scatter(
    x=df_combined_industrial_24['MES'],
    y=df_combined_industrial_24['CONS_MEDIDO'],
    mode='lines+markers',
    name='Consumo Médio (Categoria: Industrial)',
    line=dict(color='green', dash='dash')
))

# Adiciona a linha da temperatura média
fig.add_trace(go.Scatter(
    x=df_combined_industrial_24['MES'],
    y=df_combined_industrial_24['Temperatura Média'],
    mode='lines+markers',
    name='Temp. Média',
    line=dict(color='red'),
    yaxis='y2'
))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal e Temperatura Média para Categoria Industrial - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    yaxis2=dict(
        title='Temperatura (°C)',
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.5,
        y=-0.2,
        orientation="h",
        xanchor="center"
    ),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

In [ ]:
correlacao_pearson = df_combined_industrial_24['CONS_MEDIDO'].corr(df_combined_industrial_24['Temperatura Média'])

print(f"Correlação de Pearson entre Consumo Médio e Temperatura Média para a Categoria Industrial: {correlacao_pearson:.4f}")

### Consumo Médio Mensal por Categoria

In [ ]:
# Define as categorias a serem analisadas
categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'INDUSTRIAL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada categoria no gráfico
for categoria in categorias:
    consumo_categoria = df_24[df_24['CATEGORIA'] == categoria]  # Filtra os dados pela categoria
    consumo_medio_categoria = consumo_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio da categoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_categoria['MES'],
        y=consumo_medio_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio (Categoria: {categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Categoria - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Tipo de Ligação


In [ ]:
tipo_ligacao = df_24['TIPO_LIGACAO'].unique()
tipo_ligacao

In [ ]:
# Define os tipos de ligação a serem analisados
tipos_ligacao = ['Hidrometrado', 'Consumo Fixo']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada tipo de ligação no gráfico
for tipo in tipos_ligacao:
    consumo_tipo = df_24[df_24['TIPO_LIGACAO'] == tipo]  # Filtra os dados pelo tipo de ligação
    consumo_medio_tipo = consumo_tipo.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()  # Calcula o consumo médio mensal

    # Adiciona a linha do consumo médio do tipo de ligação ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_tipo['MES'],
        y=consumo_medio_tipo['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({tipo})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Tipo de Ligação - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()

### Consumo Médio Mensal por Subcategoria

In [ ]:
sub_categoria = df_24['SUB_CATEGORIA'].unique()
sub_categoria

In [ ]:
# Define as subcategorias a serem analisadas
sub_categorias = ['PUBLICA', 'COMERCIAL', 'RESIDENCIAL', 'RES. SOCIAL', 'INDUSTRIAL', 'RES. INDUBRASIL']

# Cria uma figura para o gráfico
fig = go.Figure()

# Loop para adicionar uma linha para cada subcategoria no gráfico
for sub_categoria in sub_categorias:
    consumo_sub_categoria = df_24[df_24['SUB_CATEGORIA'] == sub_categoria]  # Filtra os dados pela subcategoria
    consumo_medio_sub_categoria = consumo_sub_categoria.groupby('MES').agg({'CONS_MEDIDO': 'mean'}).reset_index()

    # Adiciona a linha do consumo médio da subcategoria ao gráfico
    fig.add_trace(go.Scatter(
        x=consumo_medio_sub_categoria['MES'],
        y=consumo_medio_sub_categoria['CONS_MEDIDO'],
        mode='lines+markers',
        name=f'Consumo Médio ({sub_categoria})'
    ))

# Configura o layout do gráfico
fig.update_layout(
    title='Consumo Médio Mensal por Subcategoria - 2024',
    xaxis=dict(title='Mês', tickmode='array', tickvals=list(range(1, 13))),
    yaxis=dict(title='Consumo Médio'),
    legend=dict(
        x=1,
        y=1,
        orientation="v",
        font=dict(size=10),
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=500
)

# Exibe o gráfico
fig.show()